In [1]:
import os
import pandas as pd
from pathlib import Path

if not "root" in locals():
    current_path = Path(os.getcwd())
    root = current_path.parent.absolute()

In [2]:
os.chdir(root)
from data_treatment.box_creator import create_binary_box

df = pd.read_csv('database.csv')
df = create_binary_box(df, threshold=2., box_length=7)

**1er modèle** : Sélection à la main des features

In [3]:
cols = ["PX_LAST_VIX_volatility",
        "VOLUME_TOTAL_CALL_VIX_volatility",
        "VOLUME_TOTAL_PUT_VIX_volatility",
        "PX_LAST_SPX_volatility",
        "PX_VOLUME_SPX_volatility",
        "PX_LAST_VSTOXX_volatility",
        "Bullish_SENTIMENT",
        "Neutral_SENTIMENT",
        "Bearish_SENTIMENT"]

hand_picked_df = df.set_index("Date")[cols + ["Box"]].dropna(axis = 0)

Essai avec régression logistique

In [4]:
import statsmodels.api as sm

X = hand_picked_df.drop(columns="Box")
y = hand_picked_df["Box"]
scaled_X = (X - X.mean())/X.std()

logit_model=sm.Logit(y,scaled_X)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.637405
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                    Box   No. Observations:                 3788
Model:                          Logit   Df Residuals:                     3779
Method:                           MLE   Df Model:                            8
Date:                Wed, 29 Dec 2021   Pseudo R-squ.:                 0.03483
Time:                        11:01:47   Log-Likelihood:                -2414.5
converged:                       True   LL-Null:                       -2501.6
Covariance Type:            nonrobust   LLR p-value:                 1.649e-33
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
PX_LAST_VIX_volatility               0.3326      0.121      2.75